## 09_11 Implement and inorder traversal with O(1) space

The direct implementation of an inorder traversal using recursion has $ O(h) $ space complexity, where **h** is the height of the tree.  Recursion can be removed with an explicit stack, but the space complexity remains $ O(h) $.

Write a nonrecursive program for computing the inorder traversal sequence for a binary tree.  Assume nodes have parent fields.

###  Subclassing binarytree.Node

I like being able to show a diagram of my tree, and [binarytree](https://anaconda.org/conda-forge/binarytree) provides that for us.  

However, we need to subclass Node, so that there is also a **parent** pointer for us to work with.

In [1]:
from binarytree import Node


class PNodeTypeError(Exception):
    """Wrong type error"""


class PNode(Node):
    """Subclass of binaryTree.Node"""
    def __init__(self, value, left=None, right=None, parent=None):
        Node.__init__(self, value, left=left, right=right)
        self.parent = parent

    def __setattr__(self, attr, obj):
        Node.__setattr__(self, attr, obj)
        if attr in ['left', 'right'] and obj is not None:
            obj.parent = self
        elif attr == 'parent':
            if obj is not None and not isinstance(obj, PNode):
                raise PNodeTypeError('Must set parent to a PNode or None')

### Now let's build our tree

In [8]:
tree = PNode(5)
tree.left = PNode(2)
tree.right = PNode(7)
tree.right.right = PNode(8)
tree.right.left = PNode(6)
tree.left.right = PNode(3)
tree.left.right.right = PNode(4)
tree.left.left = PNode(1)

### Let's visualize the tree

In [9]:
print(tree)


    ____5__
   /       \
  2         7
 / \       / \
1   3     6   8
     \
      4



### So an inorder traversal goes like this

1.  Traverse the left-subtree (recursive)
2.  Traverse the root
3.  Traverse the right-subtree (recursive)

So in our example, we want the following traversal

```1 - 2 - 3 - 4 - 5 - 6 - 7 - 8```

But we **cannot use recursion**, nor can we use an explicit stack.

So here's the approach, using a **trailing object reference**
### Until you have completely traversed the tree
#### if your previous node was a PARENT
1.  if you can go to the left
    * go to the left
2.  if you cannot go to the left
    * write out the value
    * go to the first available of right or parent


####  else if your previous node was a LEFT
1.  write out the value
2.  go to the first available of right or parent


####  otherwise
1.  go the parent



In [12]:
def solve(root):
    """This makes use of the special *or* and *is* functions
        *or* is used to get the first non-None value
        *is* is used for object equality
    """
    prev, result = None, []
    while root:
        if prev is root.parent:             # if your last node was the parent
            if root.left:                       # and you can go left
                next = root.left                    # go left
            else:                               # but if you cannot go left
                result.append(root.value)           # record the node
                next = root.right or root.parent    # next is first non-None right or parent
        elif prev is root.left:             # but if your last node was left
            result.append(root.value)           # record the node
            next = root.right or root.parent        # next is first non-None right or parent
        else:                               # and for any other csae
            next = root.parent                  # next node is parent

        # keep track of previously visited node
        prev, root = root, next
        
    return result

In [18]:
solve(tree)

[1, 2, 3, 4, 5, 6, 7, 8]

In [21]:
tree = PNode(1)
tree.left = PNode(2)
tree.left.left = PNode(3)
tree.left.left.left = PNode(4)
tree.left.left.left.left = PNode(5)

print(tree)


        1
       /
      2
     /
    3
   /
  4
 /
5



In [22]:
solve(tree)

[5, 4, 3, 2, 1]

In [23]:
tree = PNode(1)
tree.right = PNode(2)
tree.right.right = PNode(3)
tree.right.right.right = PNode(4)
tree.right.right.right.right = PNode(5)

print(tree)


1
 \
  2
   \
    3
     \
      4
       \
        5



In [24]:
solve(tree)

[1, 2, 3, 4, 5]

In [25]:
tree = PNode(1)
tree.left = PNode(2)
tree.left.right = PNode(3)
tree.left.right.left = PNode(4)
tree.left.right.left.right = PNode(5)

print(tree)


  ______1
 /
2____
     \
    __3
   /
  4
   \
    5



In [26]:
solve(tree)

[2, 4, 5, 3, 1]